In [1]:
import numpy as np 
import re
import datetime as dt 
import matplotlib.pyplot as plt 
import pandas as pd
import sys
from tqdm import tqdm   
sys.path.append('/Users/jerald/Documents/Dir/Python/stocker')
from models.sim.lsm import OptionSim as sim

def prep_df(df, rf_rate = 0.45, nsims = 100):
    df['rf']  = rf_rate
    df['nsim'] = nsims
    df['impliedvolatility'] = df.impliedvolatility.astype(float)
    df['strike'] = df.strike.astype(float)
    df['stk_price'] = df.stk_price.astype(float)
    df['lastprice'] = df.lastprice.astype(float)
    df.expiry = pd.to_datetime(df.expiry)
    df.gatherdate = pd.to_datetime(df.gatherdate)
    df['dte'] = ((df.expiry - df.gatherdate).dt.days).astype(float) 
    # df['timevalue'] = df.timevalue.astype(float) / 1e7
    df['timevalue'] = df.dte / 252
    return df


    
def lsm(df, rf_rate = 0.0379, nsims = 1000, jump = False):
    df = prep_df(df, rf_rate, nsims) 
    lodf = []
    vals = list(zip(df.stk_price, df.strike, df.rf, df.dte,df.timevalue, df.impliedvolatility,df.type, df.nsim ,df.lastprice))
    for s, k, r, dte, tv,  sig, ty, nsim, lp in tqdm(vals):
        s = sim(s, k, r, dte, tv, sig, ty, nsim, lp)
        lodf.append(s.run(jump = jump))
    
    return pd.concat(lodf)



from bin.main import Manager 
m = Manager('../../')

Options db Connected: 2024-11-13 12:11:00.723959
Prices Connected: 2024-11-13 12:11:00.724924


In [2]:
#  Simulate for an Entire Stock 
df =pd.read_sql('select * from spy where date(gatherdate) > date("now", "-1 day")', m.Optionsdb.change_db)
df = m.Optionsdb.pcdb(df) 
df['moneyness'] = df.stk_price / df.strike
df['dte'] = (pd.to_datetime(df.expiry) - pd.to_datetime(df.gatherdate)).dt.days

df = df[(df.moneyness.between(0.65, 1.35)) & (df.dte.between(10, 40)) & (df.lastprice > 0.05)].dropna()
ind = df[['contractsymbol','gatherdate']]

# Simulate For An Entire Stock 
est = lsm(df, rf_rate = 0.0379, nsims = 1000, jump = True)
est['contractsymbol'] = df.contractsymbol.values
est['avg_est'] = est[['LSMC Normal', 'LSMC Poly', 'LSMC CV', 'MC']].mean(axis = 1).round(2)
est = est.set_index('contractsymbol').reset_index()

100%|██████████| 979/979 [00:46<00:00, 20.94it/s]


In [3]:
df.contractsymbol.unique().shape

(979,)

In [4]:
est_df = m.Optionsdb.pcdb(est)[(est.Observed < est.avg_est) & (est.Observed.between(.25, .50))]
# Discount (or premium) percentage: The % difference between the option's market price and the model's price.
est_df['discount'] = ((est_df.Observed - est_df.avg_est) / est_df.Observed) * 100
est_df.sort_values('discount', ascending = True).head(20)

,contractsymbol,stock,type,strike,expiry,LSMC Normal,LSMC Poly,LSMC CV,MC,BS,Observed,avg_est,discount
636,SPY241213C00630000,SPY,Call,630.0,2024-12-13,1.874238,2.097356,2.847780,2.953374,0.969544,0.36,2.44,-577.777778
838,SPY241220C00635000,SPY,Call,635.0,2024-12-20,2.131438,1.966865,2.870070,2.798730,1.069971,0.36,2.44,-577.777778
421,SPY241206C00625000,SPY,Call,625.0,2024-12-06,1.658015,1.717566,2.470795,1.858736,0.778741,0.32,1.93,-503.125000
655,SPY241213P00500000,SPY,Put,500.0,2024-12-13,0.326286,0.727308,0.739759,0.697728,0.680075,0.31,0.62,-100.000000
661,SPY241213P00530000,SPY,Put,530.0,2024-12-13,0.498869,0.829551,1.166139,1.300642,0.934249,0.48,0.95,-97.916667
446,SPY241206P00509000,SPY,Put,509.0,2024-12-06,0.359909,0.395478,0.846638,0.603638,0.552950,0.28,0.55,-96.428571
853,SPY241220P00475000,SPY,Put,475.0,2024-12-20,0.255748,0.624026,0.742285,0.679193,0.694742,0.30,0.58,-93.333333
452,SPY241206P00515000,SPY,Put,515.0,2024-12-06,0.351631,0.536316,0.441306,0.683985,0.578671,0.26,0.50,-92.307692
851,SPY241220P00465000,SPY,Put,465.0,2024-12-20,0.264231,0.380251,0.763150,0.532044,0.625193,0.25,0.48,-92.000000
656,SPY241213P00505000,SPY,Put,505.0,2024-12-13,0.312043,0.459754,0.632146,1.028933,0.727637,0.32,0.61,-90.625000


In [30]:
est_df[est_df.discount < 0].shape[0], est_df[est_df.discount > 0].shape[0], est_df.shape[0]

(35, 0, 35)

In [26]:
df[df['amnt'] > 0 ]

,contractsymbol,stock,type,strike,expiry,gatherdate,stk_price,lastprice,ask,bid,...,iv_avg_all,vol_chg,oi_chg,flag,amnt,moneyness,dte,rf,nsim,timevalue
936,NVDA241101C00103000,NVDA,Call,103.0,2024-11-01,2024-10-18 13:42:52,138.26,34.90,36.00,35.80,...,0.621537,-3.0,12.0,1,1,1.342330,13.0,0.0379,1000,0.051587
937,NVDA241101C00104000,NVDA,Call,104.0,2024-11-01,2024-10-18 13:42:52,138.26,34.47,35.00,34.80,...,0.630687,0.0,3.0,1,2,1.329423,13.0,0.0379,1000,0.051587
943,NVDA241101C00110000,NVDA,Call,110.0,2024-11-01,2024-10-18 13:42:52,138.26,28.14,29.05,28.90,...,0.581806,-9.0,41.0,1,12,1.256909,13.0,0.0379,1000,0.051587
944,NVDA241101C00111000,NVDA,Call,111.0,2024-11-01,2024-10-18 13:42:52,138.26,27.45,28.10,27.90,...,0.573735,16.0,2.0,1,1,1.245586,13.0,0.0379,1000,0.051587
949,NVDA241101C00116000,NVDA,Call,116.0,2024-11-01,2024-10-18 13:42:52,138.26,22.45,23.15,23.05,...,0.542788,-12.0,32.0,1,17,1.191897,13.0,0.0379,1000,0.051587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,NVDA241122P00141000,NVDA,Put,141.0,2024-11-22,2024-10-18 13:42:52,138.26,11.45,11.05,10.90,...,0.526201,-17.0,22.0,1,3,0.980567,34.0,0.0379,1000,0.134921
2164,NVDA241122P00142000,NVDA,Put,142.0,2024-11-22,2024-10-18 13:42:52,138.26,12.00,11.60,11.45,...,0.521746,-5.0,15.0,1,8,0.973662,34.0,0.0379,1000,0.134921
2165,NVDA241122P00144000,NVDA,Put,144.0,2024-11-22,2024-10-18 13:42:52,138.26,12.80,12.75,12.60,...,0.518547,0.0,11.0,1,10,0.960139,34.0,0.0379,1000,0.134921
2167,NVDA241122P00150000,NVDA,Put,150.0,2024-11-22,2024-10-18 13:42:52,138.26,16.60,16.55,16.40,...,0.500884,1.0,18.0,1,2,0.921733,34.0,0.0379,1000,0.134921
